In [1]:
import os
import json
from docx import Document
from io import StringIO, BytesIO
import re

import pandas as pd

#### Define file directories

In [2]:
notebook_dir = os.getcwd()
sop_dir = '/Users/public/Desktop/SOPs/'
os.chdir(sop_dir)
sop_list = os.listdir('.')
len(sop_list)

2021

#### Formulate desired jurisdictions and unwanted files

In [3]:
juri_list = {'AB', 'BI', 'BU', 'DE', 'DFPF', 
            'NW', 'PO', 'RI', 'RM', 'SC', 
            'SQ', 'SX', 'UN', 'VA', 'WP', 
            'WV', 'DF PF'}
exclude_docs = ['- report' # gray; non-police
                , '- drugs -' # gray; non-police
                , 'bomb threats' # red; fire-related
                , 'water problem' # red; fire-related
                , 'wildfire'
                , 'stove fire'
                , 'electrical problem'
               ]

In [4]:
sop_list

['AB - 1033 - Officer in trouble.docx',
 'AB - 911.docx',
 'AB - AB911 cell.docx',
 'AB - AB911 Landline.docx',
 'AB - ABANDV - Abandoned vehicle.docx',
 'AB - ABANDV - Reports.docx',
 'AB - ABDUC - Abduction.docx',
 'AB - AIREM - Airplane Crash.docx',
 'AB - ALARM - Audible Alarm.docx',
 'AB - ALARMA - Airport alarm.docx',
 'AB - ALARMD - Domestic violence alarm.docx',
 'AB - ALARMH - Holdup alarm.docx',
 'AB - ALARMS - Silent or Panic Alarm.docx',
 'AB - ANIMAL.docx',
 'AB - ANNOY - Annoying circumstance.docx',
 'AB - ARREST.docx',
 'AB - ARSON.docx',
 'AB - ASLT - Assault.docx',
 'AB - ASLTI - Assault in Progress.docx',
 'AB - ASLTSX - Sexual assault.docx',
 'AB - ASSGP - Assist General Public.docx',
 'AB - ASSMHA - Assist Mental Health Act.docx',
 'AB - ASSOA - Assist Other Agency.docx',
 'AB - ASSPFA - Assist Police Fire Ambulance.docx',
 'AB - BAIT-Bait Car Activation.docx',
 'AB - BNE - Break and Enter.docx',
 'AB - BNEI - Break and enter in progress.docx',
 'AB - BOMB - Bomb Th

In [5]:
juris, types, fullname = list(), list(), list()
excl_juris, excl_types, excl_docs, excl_fullname = list(), list(), list(), list()
unmatched_sops = list()

sopreg = r'^([A-Z]+)[ -]*([\w ]+)-([\w ,]*).docx'

for sop in sop_list:
    isExcluded = re.search(r'|'.join(exclude_docs), sop, re.IGNORECASE)
    if isExcluded:
        unmatched_sops.append(sop)
        continue
        
    match = re.findall(r'^([A-Z]+)[ -]+([\w ]*)-([\w ,]*).docx', sop, re.IGNORECASE)
    if not match:
        unmatched_sops.append(sop)
        continue
        
    juri, acrn, doc = match[0]
    juri = re.sub(' ', '', juri.upper())
    doc = doc.upper().strip()
    acrn = acrn.upper().strip()
    if juri not in juri_list:
        excl_juris.append(juri)
        excl_types.append(acrn)
        excl_docs.append(doc)
        excl_fullname.append(sop)
        continue
        
    type_adjust = doc if len(acrn) == 0 else acrn
    juris.append(juri)
    types.append(type_adjust)
    fullname.append(sop)
    
sop_docs = pd.DataFrame({'juri': juris, 'type': types, 'filename': fullname})

sop_unmatched = pd.DataFrame({'filename': unmatched_sops})

sop_excludedjuri = pd.DataFrame({'juri': excl_juris, 'type': excl_docs, 'filename': excl_fullname})


In [6]:
sop_docs

,juri,type,filename
0,AB,1033,AB - 1033 - Officer in trouble.docx
1,AB,911,AB - 911.docx
2,AB,AB911 CELL,AB - AB911 cell.docx
3,AB,AB911 LANDLINE,AB - AB911 Landline.docx
4,AB,ABANDV,AB - ABANDV - Abandoned vehicle.docx
...,...,...,...
1621,WV,TRAFFS,WV - TRAFFS - Traffic Stop.docx
1622,WV,UNWANT,WV - UNWANT - Unwanted person.docx
1623,WV,WARRAN,WV - WARRAN - Warrant.docx
1624,WV,WEAPON,WV - WEAPON.docx


In [7]:
type_to_juri = sop_docs \
    .groupby('type') \
    .agg({'juri': lambda x: list(x), 'filename': lambda x: list(x)}) \
    .reset_index()
type_to_juri.loc[:, 'juri_count'] = type_to_juri.loc[:, 'juri'].apply(len)
type_to_juri = type_to_juri.sort_values(by = 'juri_count', ascending = False, ignore_index = True)
type_to_juri

,type,juri,filename,juri_count
0,1033,"[AB, BI, BU, DE, DFPF, NW, PO, RI, RM, SC, SQ,...","[AB - 1033 - Officer in trouble.docx, BI - 103...",16
1,DOMI,"[AB, BI, BU, DE, DFPF, NW, PO, RI, RM, SC, SQ,...","[AB - DOMI - Domestic in progress.docx, BI - D...",16
2,FOUNDP,"[AB, BI, BU, DE, DFPF, NW, PO, RI, RM, SC, SQ,...","[AB - FOUNDP - Found Person.docx, BI - FOUNDP ...",16
3,HAZARD,"[AB, BI, BU, DE, DFPF, NW, PO, RI, RM, SC, SQ,...","[AB - HAZARD - Hazard.docx, BI - HAZARD - Haza...",16
4,IMPAIR,"[AB, BI, BU, DE, DFPF, NW, PO, RI, RM, SC, SQ,...","[AB - IMPAIR - Impaired driver.docx, BI - IMPA...",16
...,...,...,...,...
123,ALARMA,"[AB, RI]","[AB - ALARMA - Airport alarm.docx, RI - ALARMA...",2
124,BORDR,"[AB, DE]","[AB - BORDR - Border Runner.docx, DE - BORDR -...",2
125,FBOAT,[VA],[VA - FBOAT - Fireboat call.docx],1
126,ELEC12,[NW],[NW - ELEC12.docx],1


In [8]:
type_to_juri['juri_count'].unique()

array([16, 15, 14,  5,  2,  1], dtype=int64)

In [9]:
type_to_juri_ok = type_to_juri[type_to_juri['juri_count'] >= 13]
type_to_juri_5 = type_to_juri[type_to_juri['juri_count'] == 5]
type_to_juri_2 = type_to_juri[type_to_juri['juri_count'] == 2]
type_to_juri_1 = type_to_juri[type_to_juri['juri_count'] == 1]

In [10]:
type_to_juri_ok['type'].unique()

array(['1033', 'DOMI', 'FOUNDP', 'HAZARD', 'IMPAIR', '911', 'INTELL',
       'JUMPER', 'MAND', 'MISCH', 'MISCHI', 'MVI', 'MVIHR', 'MVIINJ',
       'OCC', 'PANHA', 'PROP', 'PURSUE', 'RECVEH', 'SCREAM', 'SHOTS',
       'SIP', 'SUDDEN', 'SUICID', 'SUSPC', 'SUSPP', 'THEFTV', 'THREATS',
       'TRAFF', 'TRAFFS', 'WEAPON', 'DRUGS', 'INSEC', 'ANIMAL', 'ASSOA',
       'ABANDV', 'ARREST', 'ARSON', 'MVIPOL', 'PARK', 'ANNOY', 'OVERD',
       'NOK', 'BREACH', 'PROWL', 'ASLT', 'MISSIP', 'MISSIC', 'ASLTI',
       'ASLTSX', 'PROST', 'ALARMS', 'ASSGP', 'THEFT', 'AB911 CELL',
       'WARRAN', 'UNWANT', 'AB911 LANDLINE', 'ABDUC', 'THEFTI', 'SUSPV',
       'ROBB', 'AIREM', 'ALARM', 'STALK', 'SHOPL', 'ALARMH', 'ROBBI',
       'MARINE', 'YOUTH', 'LIQUOR', 'COUNTERFEIT', 'EXPLOS', 'EXTORT',
       'BNEI', 'DOMRPT', 'FIGHT', 'FIREAR', 'BOMB', 'BNE', 'FRAUD',
       'DISTB', 'HARASS', 'DEMON', 'ASSPFA', 'HOMEIN', 'HOSTAG', 'BAIT',
       'CHECK', 'BYLAW', 'INDEC', 'KPEACE', 'INDUST', 'ASSMHA', 'SPAT',
       

In [11]:
os.chdir(notebook_dir)
type_to_juri_ok.to_csv('../data/interim/sop_types_valid.csv', index = False)
type_to_juri_excluded = type_to_juri[type_to_juri['juri_count'] < 13]
type_to_juri_excluded.to_csv('../data/interim/sop_types_excluded.csv', index = False)
sop_excludedjuri.to_csv('../data/interim/sop_excluded_due_to_juris.csv', index = False)
sop_unmatched.to_csv('../data/interim/sop_excluded_due_to_type.csv', index = False)

In [12]:
# Added by Jake

def para_to_text(p):
    """
    A function to find every texts in the paragraph

    params
    ----
    p : docx.Document.Paragraph object

    returns
    ----
    str 

    """
    rs = p._element.xpath(".//w:t")
    return u"".join([r.text for r in rs])


In [13]:
def parse_docx_role(filepath):
    with open(filepath, 'rb') as f:
        source_stream = BytesIO(f.read())
    f.close()
    doc = Document(source_stream)
    paras = doc.paragraphs
    res = dict()
    role = None
    texts, level = list(), list()
    for p in paras:
        style = p.style.name
        text = para_to_text(p).strip()
        
        if len(text) == 0:
            continue
        if style == 'Heading 1':
            role = re.findall(r'([\w ]+)', text.lower())[0]
            res[role] = list()
        elif role is not None:
            res[role].append(text)
    return res

> Because the SOP documents have inconsistent file structure, I will parse on the event type level

> For example, the "1033" event type doesn't have situations
![](img/sop_doc_inconsistent_level.PNG)

In [14]:
def parse_one_event(curr_row):
    cwd = os.getcwd()
    os.chdir(sop_dir)
    event_type = curr_row['type']
    jsons = dict()
    for juri, file in zip(curr_row['juri'], curr_row['filename']):
        jsons[juri] = parse_docx_role(file)
    os.chdir(cwd)
    return jsons

In [15]:
num_rows = type_to_juri_ok.shape
num_rows

(97, 4)

In [16]:
os.chdir(notebook_dir)

In [17]:
for i in range(num_rows[0]):
    row = type_to_juri_ok.iloc[i, :]
    event = parse_one_event(row)
    doctype = row['type']
    outfile_name = f'../data/sop_jsons/{doctype}.txt'
    with open(outfile_name, 'w') as outfile:
        json.dump(event, outfile)
    outfile.close()

In [18]:
# with open('../data/sop_jsons/PROP.txt') as f:
#     jj = json.load(f)
# print(json.dumps(jj, indent = 4))

In [19]:
def parse_docx_role_df(filepath):
    with open(filepath, 'rb') as f:
        source_stream = BytesIO(f.read())
    f.close()
    doc = Document(source_stream)
    paras = doc.paragraphs
    res = dict()
    role = None
    texts, level = list(), list()
    for p in paras:
        style = p.style.name
        text = para_to_text(p)
        texts.append(text)
        level.append(style)
    return pd.DataFrame({'text': texts, 'level': level})

In [20]:
os.getcwd()
os.chdir(sop_dir)

In [21]:
os.chdir(sop_dir)
parse_docx_role_df('BU - TRAFFS - Traffic Stop.docx')[:30]

,text,level
0,This SOP covers the TRAFFS – Traffic suspensio...,Normal
1,call taker,Heading 1
2,Do not use,Style1
3,Dispatcher,Heading 1
4,Create a call,Style1
5,Record,Style1
6,Start time,Style1
7,Vehicle details,Style1
8,Impoundment information and duration,Style1
9,Type of suspension,Style1


In [22]:
def parse_docx_situation_df(doc_type, juri, filepath):
    with open(sop_dir + filepath, 'rb') as f:
        source_stream = BytesIO(f.read())
    f.close()
    doc = Document(source_stream)
    paras = doc.paragraphs
    res = pd.DataFrame()
    role, situation = None, ''
    roles, situations, texts = list(), list(), list()
    types, juris, files = list(), list(), list()
    for p in paras:
        style = p.style.name
        text = para_to_text(p)
        if re.search(r'type is not used', text.lower()):
            return res
        if len(text) == 0:
            continue
        if style == 'Normal':
            role, situation = None, ''
        elif style == 'Heading 1':
            role = re.findall(r'([\w ]+)', text.lower())[0].strip()
#             print(role)
            situation = ''
        elif style == 'Heading 2':
            situation = text.strip()
        elif role is not None:
            text = text.strip()
            roles.append(role)
            situations.append(situation)
            texts.append(text)
            
    res = pd.DataFrame({'role': roles, 'situation': situations, 'sop': texts}) \
        .groupby(['role', 'situation']) \
        .agg({'sop': lambda x: list(x)}) \
        .reset_index()
    res['juri'] = juri
    res['type'] = doc_type
    res['filename'] = filepath
    res = res[['type', 'juri', 'role', 'situation', 'sop', 'filename']]
    return res

In [23]:
def parse_docx_situation_df2(doc_type, juri, filepath):
    with open(sop_dir + filepath, 'rb') as f:
        source_stream = BytesIO(f.read())
    f.close()
    doc = Document(source_stream)
    paras = doc.paragraphs
    res = pd.DataFrame()
    role, situation = None, ''
    roles, situations, texts = list(), list(), list()
    types, juris, files = list(), list(), list()
    for p in paras:
        style = p.style.name
        text = para_to_text(p)
#         if re.search(r'type is not used', text.lower()):
#             return res
        if len(text) == 0:
            continue
        if style == 'Normal':
#             role, situation = None, ''
            continue
        elif style == 'Heading 1':
            role = re.findall(r'([\w ]+)', text.lower())[0].strip()
#             print(role)
            situation = ''
        elif style == 'Heading 2':
            situation = text.strip()
        elif role is not None:
            text = text.strip()
            roles.append(role)
            situations.append(situation)
            texts.append(text)
            
    res = pd.DataFrame({'role': roles, 'situation': situations, 'sop': texts}) \
        .groupby(['role', 'situation']) \
        .agg({'sop': lambda x: list(x)}) \
        .reset_index()
    res['juri'] = juri
    res['type'] = doc_type
    res['filename'] = filepath
    res = res[['type', 'juri', 'role', 'situation', 'sop', 'filename']]
    return res

In [24]:
parse_docx_situation_df2('TRAFFS', 'BU', 'BU - TRAFFS - Traffic Stop.docx')

,type,juri,role,situation,sop,filename
0,TRAFFS,BU,call taker,,[Do not use],BU - TRAFFS - Traffic Stop.docx
1,TRAFFS,BU,dispatcher,,"[Create a call, Record, Start time, Vehicle de...",BU - TRAFFS - Traffic Stop.docx
2,TRAFFS,BU,dispatcher,Vehicle impoundment,"[Create a call, Assign the member]",BU - TRAFFS - Traffic Stop.docx


In [25]:
def get_all_situations(df):
    df_all_situation = pd.DataFrame()
    for i in range(len(df)):
        row = df.iloc[i, :]
        doc_type = row['type']
        for juri, filename in zip(row['juri'], row['filename']):
            situ_df = parse_docx_situation_df(doc_type, juri, filename)
            df_all_situation = df_all_situation.append(situ_df)
    return df_all_situation.reset_index(drop = True)

In [26]:
def get_all_situations2(df):
    df_all_situation = pd.DataFrame()
    for i in range(len(df)):
        row = df.iloc[i, :]
        doc_type = row['type']
        for juri, filename in zip(row['juri'], row['filename']):
            situ_df = parse_docx_situation_df2(doc_type, juri, filename)
            df_all_situation = df_all_situation.append(situ_df)
    return df_all_situation.reset_index(drop = True)

In [27]:
type_to_juri_ok

,type,juri,filename,juri_count
0,1033,"[AB, BI, BU, DE, DFPF, NW, PO, RI, RM, SC, SQ,...","[AB - 1033 - Officer in trouble.docx, BI - 103...",16
1,DOMI,"[AB, BI, BU, DE, DFPF, NW, PO, RI, RM, SC, SQ,...","[AB - DOMI - Domestic in progress.docx, BI - D...",16
2,FOUNDP,"[AB, BI, BU, DE, DFPF, NW, PO, RI, RM, SC, SQ,...","[AB - FOUNDP - Found Person.docx, BI - FOUNDP ...",16
3,HAZARD,"[AB, BI, BU, DE, DFPF, NW, PO, RI, RM, SC, SQ,...","[AB - HAZARD - Hazard.docx, BI - HAZARD - Haza...",16
4,IMPAIR,"[AB, BI, BU, DE, DFPF, NW, PO, RI, RM, SC, SQ,...","[AB - IMPAIR - Impaired driver.docx, BI - IMPA...",16
...,...,...,...,...
92,INDUST,"[AB, BI, BU, DE, NW, PO, RI, RM, SC, SQ, SX, U...","[AB - INDUST - Industrial Accident.docx, BI - ...",15
93,ASSMHA,"[AB, BI, BU, DE, NW, PO, RI, RM, SC, SQ, SX, U...","[AB - ASSMHA - Assist Mental Health Act.docx, ...",15
94,SPAT,"[BI, BU, DE, NW, PO, RI, RM, SC, SQ, SX, UN, V...",[BI - SPAT - Special Attention Detail Event.do...,14
95,ALARMD,"[AB, BI, BU, DE, NW, PO, RI, RM, SC, SQ, SX, V...","[AB - ALARMD - Domestic violence alarm.docx, B...",14


In [28]:

# type_to_juri_ok = pd.read_csv(
#     notebook_dir + '/../data/interim/sop_types_valid.csv',
#     keep_default_na = False, 
#     converters = {'juri': eval, 'filename': eval}
# )

In [29]:
df_all_situation = get_all_situations2(type_to_juri_ok)

In [30]:
df_all_situation[ (df_all_situation['type'] == 'TRAFFS') 
                & (df_all_situation['role'] == 'call taker')]

,type,juri,role,situation,sop,filename
2782,TRAFFS,AB,call taker,,[event type is not used],AB - TRAFFS - Traffic Stop.docx
2785,TRAFFS,BI,call taker,,[Call type is not used],BI - TRAFFS - Traffic Stop.docx
2788,TRAFFS,BU,call taker,,[Do not use],BU - TRAFFS - Traffic Stop.docx
2791,TRAFFS,DE,call taker,,[event type is not used],DE - TRAFFS - Traffic Stop.docx
2794,TRAFFS,DFPF,call taker,,[Call type is not used],DFPF - TRAFFS - Traffic Stop.docx
2797,TRAFFS,NW,call taker,,[Call type is not used],NW - TRAFFS - Traffic Stop.docx
2800,TRAFFS,PO,call taker,,[Call type is not used],PO - TRAFFS - Traffic Stop.docx
2803,TRAFFS,RI,call taker,,[Call type is not used],RI - TRAFFS - Traffic Stop.docx
2806,TRAFFS,RM,call taker,,[Call type is not used],RM - TRAFFS - Traffic Stop.docx
2809,TRAFFS,SC,call taker,,[Call type is not used],SC - TRAFFS - Traffic Stop.docx


In [31]:
os.chdir(notebook_dir)
df_all_situation.to_csv('../data/interim/all_situation.csv', index = False)

In [32]:
df_call = df_all_situation[ 
    (df_all_situation['role'] == 'call taker') 
    & (df_all_situation['situation'].str.len() > 0)
].reset_index(drop = True)
df_call

,type,juri,role,situation,sop,filename
0,1033,DE,call taker,CBSA alarm policy,"[Listen to alarm, Acknowledge the alarm by pre...",DE - 1033 - Officer in trouble.docx
1,DOMI,AB,call taker,All other DOMI reports,[Create a call in every instance. Do not cance...,AB - DOMI - Domestic in progress.docx
2,DOMI,AB,call taker,GPS Panic Alarms,"[Create a call, Remain on the line recording a...",AB - DOMI - Domestic in progress.docx
3,FOUNDP,AB,call taker,Address Obtained,"[Run address on QBA:, If the person is negativ...",AB - FOUNDP - Found Person.docx
4,FOUNDP,AB,call taker,Name or medic alert is known,"[Run name on CPIC, If the person is negative o...",AB - FOUNDP - Found Person.docx
...,...,...,...,...,...,...
3710,DNA,SX,call taker,DNA Warrant:,[See WARRAN (Warrants) SOP],SX - DNA - DNA collection.docx
3711,DNA,UN,call taker,DNA Warrant:,[See WARRAN (Warrants) SOP],UN - DNA - DNA collection.docx
3712,DNA,VA,call taker,DNA Warrant:,[See WARRAN (Warrants) SOP],VA - DNA - DNA collection.docx
3713,DNA,WP,call taker,DNA Warrant:,[See WARRAN (Warrants) SOP],WP - DNA - DNA collection.docx


In [33]:
df_call.to_csv('../data/interim/calltaker_situation.csv', index = False)

In [34]:
df_dispatcher = df_all_situation[ 
    (df_all_situation['role'] == 'dispatcher') 
    & (df_all_situation['situation'].str.len() > 0)
].reset_index(drop = True)
df_call.to_csv('../data/interim/dispatcher_situation.csv', index = False)
df_dispatcher


,type,juri,role,situation,sop,filename
0,1033,AB,dispatcher,1033 activations by member over the radio,[Refer to Instructions – Emergency Key Activat...,AB - 1033 - Officer in trouble.docx
1,1033,BI,dispatcher,1033 activations by member over the radio,[Refer to Instructions – Emergency Key Activat...,BI - 1033 - Officer in trouble.docx
2,1033,BU,dispatcher,1033 activations by member over the radio,[Refer to Instructions – Emergency Key Activat...,BU - 1033 - Officer in trouble.docx
3,1033,DE,dispatcher,1033 activations by member over the radio,[Refer to Instructions – Emergency Key Activat...,DE - 1033 - Officer in trouble.docx
4,1033,DE,dispatcher,CBSA alarm,[If the CBSA alarm key rings on your phone set...,DE - 1033 - Officer in trouble.docx
...,...,...,...,...,...,...
3073,SPAT,WP,dispatcher,"Government Building, Agency, Consulate or Othe...","[Assign a member, It is the responsibility of ...",WP - SPAT - Special Attention Detail Event.docx
3074,SPAT,WV,dispatcher,Commercial Premises,"[Assign a member, It is the responsibility of ...",WV - SPAT - Special Attention Detail Event.docx
3075,SPAT,WV,dispatcher,"Government Building, Agency, Consulate or Othe...","[Assign a member, It is the responsibility of ...",WV - SPAT - Special Attention Detail Event.docx
3076,DNA,NW,dispatcher,Person attends front counter or member request...,"[Canvas for DNA certified member, Contact Watc...",NW - DNA - DNA collection .docx
